<center><h1 style="color:#173F8A;"> Metodos para Ecuaciones Diferenciales, IMT3410, 2022-2 </h1></center>
<h3 style="color:#173F8A;text-align:right;"> Instituto de Ingenieria Matematica y Computacional<br>  Pontificia Universidad Catolica de Chile <br>  </h3>
<h3 style="color:#0176DE;text-align:right;"> Profesor. Manuel A. Sanchez<br> </h3>
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC: celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 -->
<!--
<figure>
<img align ="right" src="IMClogo.png" alt="logo" width="250" height="400"><br><br><br><br><br>
</figure>
 -->

<h2 style="color:#03122E;text-align:center;"> Capitulo 2. Metodos para Ecuaciones Diferenciales Parciales Elipticas<br> </h2>
<h3 style="color:#03122E;text-align:center;">             Metodos de Elementos Finitos Mixtos (Mixed Methods) <br> </h3>
<h3 style="color:#03122E;text-align:center;">             Ecuacion de Poisson <br> </h3>
<hr style="border:3px solid #E3AE00 "> </hr>

In [14]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw
#import netgen.gui

# Domain geometry
geo = unit_square
# Generate mesh with meshsize
h = .2
mesh = Mesh(geo.GenerateMesh(maxh=h))

# Manufactured solution
uexact = CoefficientFunction( 16*x*(1-x)*y*(1-y) )
qexact = CoefficientFunction((-16*(1-2*x)*y*(1-y), -16*(1-2*y)*x*(1-x)  ))
# Source according 
f      = 32*y*(1-y)+32*x*(1-x)
Draw(uexact, mesh, 'solucion exacta u ')
Draw(qexact, mesh, 'solucion exacta q')
#print(mesh.GetBoundaries())

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

# DG method

Consideremos el problema eliptico
\begin{equation}
\begin{array}{rclll}
\displaystyle
-\mbox{div}(c \nabla u) & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & u_D & \mbox{sobre} & \partial \Omega_{D} \\
\displaystyle c \nabla u\cdot n & = & g_N & \mbox{sobre} & \partial \Omega_{N} \\
\end{array}
\end{equation}

Reescribimos la ecuacion diferencial como un sistema de primer orden introduciendo el flujo $q$, y $a = c^{-1}$
\begin{equation}
\begin{array}{rclll}
\displaystyle a q & = & \nabla u & \mbox{en} & \Omega \\
\displaystyle
\mbox{div}( q ) & = & f & \mbox{en} & \Omega \\
\displaystyle u & = & u_D & \mbox{sobre} & \partial \Omega_{D} \\
\displaystyle q\cdot n & = & -g_N & \mbox{sobre} & \partial \Omega_{N} \\
\end{array}
\end{equation}


Entonces la formulacion variacional mixta (busca dos soluciones) queda: Hallar $(q,u)$ en cada elemento $K\in \mathcal T_h$

\begin{equation}
\begin{array}{rcl}
\displaystyle
\int_{K} a q \cdot r\,dx + \int_{K} u\,\mbox{div}(r)\,dx & = &
\displaystyle 
-\int_{\partial K} u r \cdot n_K ds \\
\displaystyle
\int_{K} q\cdot \nabla v\,dx & = & 
\displaystyle
 \int_{\Omega} f v+\int_{\partial K} v q*n_K \,ds \,dx
\end{array}
\end{equation}

Espacios de Galerkin discontinuo:
\begin{equation}
V_h = \{v\in L^{2}(\Omega): \forall K\in \mathcal T_h, v|_{K}\in \mathcal P^{k}\},\quad
\Sigma_h = \{ \}
\end{equation}

Formulacion de discontinuous Galerkin (mixta):



In [16]:
# Mixed form
order = 3
Wh = L2(mesh, order=order, dgjumps=True)
Vh = VectorL2(mesh, order=order, dgjumps=True)
fes = FESpace([Vh,Wh] )

# Trial and test functions
q,u = fes.TrialFunction()
v,w = fes.TestFunction()

# Numerical fluxes
n = specialcf.normal(2)
jump_u = (u-u.Other())*n
jump_w = (w-w.Other())*n
jump_q = (q-q.Other())*n
jump_v = (v-v.Other())*n

mean_u = 0.5*(u+u.Other())
mean_w = 0.5*(w+w.Other())
mean_q = 0.5*(q+q.Other())
mean_v = 0.5*(v+v.Other())

eta = max(4*order**2, 4)
beta = CoefficientFunction((1.0,1.0))
h = specialcf.mesh_size
phi_u_i = mean_u - beta*jump_u
phi_q_i = mean_q + beta*jump_q - eta/h*jump_u 
phi_u_b = 0
phi_q_b = q - eta/h*u*n 

dS = dx(element_boundary=True)
a_dg = BilinearForm(fes)
# first equation
a_dg += (q*v + u*div(v) )* dx 
a_dg += -phi_u_b * v*n*ds(skeleton=True)
a_dg += -phi_u_i*jump_v * dx(skeleton=True)
# second equation
a_dg += (q*grad(w))* dx 
a_dg += -(phi_q_i*jump_w)*dx(skeleton=True)
a_dg += -(phi_q_b*n*w)*ds(skeleton=True)
a_dg.Assemble()

fdg = LinearForm(fes)
fdg += f*w*dx 
fdg.Assemble()

gf = GridFunction(fes)
gfq, gfu = gf.components
gf.vec.data = a_dg.mat.Inverse() * fdg.vec
Draw (gfu)
Draw (gfq)

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene